In [1]:
import csv
import sqlite3
import numpy as np
import re
import os
import json
import pandas as pd

In [2]:
#店家table
district_mapping = {'Neihu':'內湖區','內湖區':'內湖區','Xinyi':'信義區','信義區':'信義區','Da’an':'大安區','大安區':'大安區',
                    'Wenshan':'文山區','文山區':'文山區','Nangang':'南港區','南港區':'南港區','Zhongzheng':'中正區','中正區':'中正區',
                    'Zhongshan':'中山區','中山區':'中山區','Beitou':'北投區','北投區':'北投區','Shilin':'士林區','士林區':'士林區',
                    'Wanhua':'萬華區','萬華區':'萬華區','Neihu':'內湖區','內湖區':'內湖區','Songshan':'松山區','松山區':'松山區',
                    'Datong':'大同區','大同區':'大同區','Xindian':'新店區','新店區':'新店區','Sanchong':'三重區','三重區':'三重區'
                   }
city_mapping = {'Taipei':'台北市','台北市':'台北市','New':'新北市','新北市':'新北市'}
spe_name = {'小鮮肉涮涮屋':'小鮮肉涮涮屋','橘色涮涮屋':'橘色涮涮屋','胡同裏的寬巷子':'胡同裏的寬巷子','好食多涮涮屋':'好食多涮涮屋',
            '火鍋106':'火鍋106','A-KUAN':'A-KUAN',
            '雞湯大叔':'雞湯大叔','大呼過癮':'大呼過癮','火鍋世家':'火鍋世家',
            '鍋真讚':'鍋真讚','天鍋宴':'天鍋宴','黑毛屋':'黑毛屋','鍋老大':'鍋老大',
            '鍋大娘':'鍋大娘','涮嘴':'涮嘴','鍋吉拉':'鍋吉拉','涮乃葉':'涮乃葉',
            '鍋膳':'鍋膳','無鍋不樂':'無鍋不樂','涮乃葉':'涮乃葉','鍋士無雙':'鍋士無雙',
            '鍋博士':'鍋博士','鍋媽媽':'鍋媽媽','川之流':'川之流','鍋董':'鍋董','鍋爸':'鍋爸',
            '【幕間一鍋':'幕間一鍋','Hi-Q':'Hi-Q褐藻生活館x鱻食','鍋HoMe':'鍋HoMe'}

pre = ['胡同裏的寬巷子','小鮮肉涮涮屋','橘色涮涮屋','A-KUAN','火鍋106','雞湯大叔','大呼過癮','火鍋世家','涮乃葉',
       '鍋大娘','鍋吉拉','鍋真讚','天鍋宴','黑毛屋','鍋老大','涮嘴',
       '鍋膳','無鍋不樂','涮乃葉','鍋博士','鍋媽媽','川之流','好食多涮涮屋','鍋董','鍋士無雙','鍋爸','【幕間一鍋','Hi-Q','鍋HoMe']


tel, addr, comment_num, price_level, store_name, brand, city, district, avg_rat = [],[],[],[],[],[],[],[],[]
for k in os.listdir('./json'):
    if k!= '.ipynb_checkpoints':
        with open('./json/{}'.format(k)) as f:
            data = json.load(f)
    for i in range(len(data)):
        try:
            tel.append(data[i]['result']['formatted_phone_number'])
            addr.append(data[i]['result']['formatted_address'])
            try:
                comment_num.append(data[i]['result']['user_ratings_total'])
            except:
                comment_num.append(None)
            try:
                price_level.append(data[i]['result']['price_level'])
            except:
                price_level.append(None)
            store_name.append(data[i]['result']['name'])
            try:
                avg_rat.append(data[i]['result']['rating'])
            except:
                avg_rat.append(None)

            for j in range(len(data[i]['result']['address_components'])):
                if data[i]['result']['address_components'][j]['types'] == ['administrative_area_level_3', 'political']:
                    district.append(district_mapping[data[i]['result']['address_components'][j]['long_name'].split(' ')[0]])
                elif data[i]['result']['address_components'][j]['types'] == ['administrative_area_level_1', 'political']:
                    city.append(city_mapping[data[i]['result']['address_components'][j]['long_name'].split(' ')[0]])

            if data[i]['result']['name'][:7] in pre:
                 brand.append(spe_name[data[i]['result']['name'][:7]])
            elif data[i]['result']['name'][:6] in pre:
                 brand.append(spe_name[data[i]['result']['name'][:6]])
            elif data[i]['result']['name'][:5] in pre:
                 brand.append(spe_name[data[i]['result']['name'][:5]])
            elif data[i]['result']['name'][:4] in pre:
                brand.append(spe_name[data[i]['result']['name'][:4]])
            elif data[i]['result']['name'][:3] in pre:
                brand.append(spe_name[data[i]['result']['name'][:3]])
            elif data[i]['result']['name'][:2] in pre:
                brand.append(spe_name[data[i]['result']['name'][:2]])

            else:
                name = data[i]['result']['name'].split('鍋')
                if len(name) == 1:
                    brand.append(re.split('/|【|-|,|;', name[0])[0])
                else:
                    brand.append(re.split('/|【|-|,|;', name[0])[0]+'鍋')
        except:
            
            continue

store = {
    "電話": tel,
    "地址": addr,
    "評論數": comment_num,
    "價錢level": price_level,
    "店名": store_name,
    "品牌": brand,
    "縣市": city,
    "地區": district,
    "平均星數": avg_rat
}
df_store = pd.DataFrame(store)

In [3]:
df_store

,電話,地址,評論數,價錢level,店名,品牌,縣市,地區,平均星數
0,02 2766 2966,"No. 251號, Yongji Rd, Xinyi District, Taipei Ci...",602.0,2.0,柳村火鍋,柳村火鍋,台北市,信義區,4.5
1,02 8785 1207,"No. 613號, Section 5 of Zhongxiao East RoadSect...",974.0,2.0,小鮮肉涮涮屋,小鮮肉涮涮屋,台北市,信義區,4.6
2,02 2723 7708,"110, Taiwan, Taipei City, Xinyi District, Song...",125.0,NaN,小川鍋物 littlerriver｜信義店｜小農蔬菜火鍋,小川鍋,台北市,信義區,4.0
3,02 2345 6988,"110, Taiwan, Taipei City, Xinyi District, Song...",1724.0,NaN,我樂鍋物Garaku Nabe Neo19 信義旗艦店( 台北推薦火鍋|信義區美食|和牛火鍋...,我樂鍋,台北市,信義區,5.0
4,02 2726 2095,"No. 20號, Lane 524, Section 5, Zhongxiao E Rd, ...",98.0,1.0,咱鉄賣鍋 台北小火鍋 信義區小火鍋 台北美食 信義區美食 台北宵夜 信義區宵夜,咱鉄賣鍋,台北市,信義區,4.4
...,...,...,...,...,...,...,...,...,...
627,02 2555 7606,"No. 125號, Zhengzhou Rd, Datong District, Taipe...",266.0,1.0,鍋HoMe即享鍋,鍋HoMe,台北市,大同區,4.5
628,02 2558 9991,"103, Taiwan, Taipei City, Datong District, Nan...",496.0,NaN,本鼎堂台式漢方麻辣鍋,本鼎堂台式漢方麻辣鍋,台北市,大同區,4.5
629,02 2597 9847,"No. 49, Section 3, Yanping N Rd, Datong Distri...",516.0,1.0,寶神日式涮涮鍋 延平店/延三夜市,寶神日式涮涮鍋,台北市,大同區,4.3
630,02 2595 7611,"No. 302號, Dalong St, Datong District, Taipei C...",295.0,NaN,5味大龍臭臭鍋,5味大龍臭臭鍋,台北市,大同區,3.9


In [4]:
#區域table 
import json
city = []
district = []
district_mapping = {'Neihu':'內湖區','內湖區':'內湖區','Xinyi':'信義區','信義區':'信義區','Da’an':'大安區','大安區':'大安區',
                    'Wenshan':'文山區','文山區':'文山區','Nangang':'南港區','南港區':'南港區','Zhongzheng':'中正區','中正區':'中正區',
                    'Zhongshan':'中山區','中山區':'中山區','Beitou':'北投區','北投區':'北投區','Shilin':'士林區','士林區':'士林區',
                    'Wanhua':'萬華區','萬華區':'萬華區','Neihu':'內湖區','內湖區':'內湖區','Songshan':'松山區','松山區':'松山區',
                    'Datong':'大同區','大同區':'大同區','Xindian':'新店區','新店區':'新店區','Sanchong':'三重區','三重區':'三重區'
                   }
city_mapping = {'Taipei':'台北市','台北市':'台北市','New':'新北市','新北市':'新北市'}
for k in os.listdir('./json'):
    if k!= '.ipynb_checkpoints':
        with open('./json/{}'.format(k)) as f:
            data = json.load(f)
    for i in range(len(data)):
        for j in range(len(data[i]['result']['address_components'])):
            if data[i]['result']['address_components'][j]['types'] == ['administrative_area_level_3', 'political']:
                district.append(district_mapping[data[i]['result']['address_components'][j]['long_name'].split(' ')[0]])
            if data[i]['result']['address_components'][j]['types'] == ['administrative_area_level_1', 'political']:
                city.append(city_mapping[data[i]['result']['address_components'][j]['long_name'].split(' ')[0]])
tmp = list(zip(city, district))
tmp = list(set(tmp))


region = {
    "縣市": [tmp[i][0] for i in range(len(tmp))],
    "地區": [tmp[i][1] for i in range(len(tmp))],
}
df_region = pd.DataFrame(region)

In [5]:
df_region

,縣市,地區
0,台北市,中山區
1,台北市,萬華區
2,台北市,士林區
3,新北市,新店區
4,台北市,中正區
5,台北市,內湖區
6,台北市,南港區
7,台北市,大安區
8,台北市,松山區
9,台北市,大同區


In [6]:
#品牌table
#特殊name
spe_name = {'小鮮肉涮涮屋':'小鮮肉涮涮屋','橘色涮涮屋':'橘色涮涮屋','胡同裏的寬巷子':'胡同裏的寬巷子','好食多涮涮屋':'好食多涮涮屋',
            '火鍋106':'火鍋106','A-KUAN':'A-KUAN',
            '雞湯大叔':'雞湯大叔','大呼過癮':'大呼過癮','火鍋世家':'火鍋世家',
            '鍋真讚':'鍋真讚','天鍋宴':'天鍋宴','黑毛屋':'黑毛屋','鍋老大':'鍋老大',
            '鍋大娘':'鍋大娘','涮嘴':'涮嘴','鍋吉拉':'鍋吉拉','涮乃葉':'涮乃葉',
            '鍋膳':'鍋膳','無鍋不樂':'無鍋不樂','涮乃葉':'涮乃葉','鍋士無雙':'鍋士無雙',
            '鍋博士':'鍋博士','鍋媽媽':'鍋媽媽','川之流':'川之流','鍋董':'鍋董','鍋爸':'鍋爸',
            '【幕間一鍋':'幕間一鍋','Hi-Q':'Hi-Q褐藻生活館x鱻食','鍋HoMe':'鍋HoMe'}

pre = ['胡同裏的寬巷子','小鮮肉涮涮屋','橘色涮涮屋','A-KUAN','火鍋106','雞湯大叔','大呼過癮','火鍋世家','涮乃葉',
       '鍋大娘','鍋吉拉','鍋真讚','天鍋宴','黑毛屋','鍋老大','涮嘴',
       '鍋膳','無鍋不樂','涮乃葉','鍋博士','鍋媽媽','川之流','好食多涮涮屋','鍋董','鍋士無雙','鍋爸','【幕間一鍋','Hi-Q','鍋HoMe']
brand = []

for k in os.listdir('./json'):
    if k!= '.ipynb_checkpoints':
        with open('./json/{}'.format(k)) as f:
            data = json.load(f)
    for i in range(len(data)):
        try:
            test = data[i]['result']['formatted_phone_number']
            if data[i]['result']['name'][:7] in pre:
                 brand.append(spe_name[data[i]['result']['name'][:7]])
            elif data[i]['result']['name'][:6] in pre:
                 brand.append(spe_name[data[i]['result']['name'][:6]])
            elif data[i]['result']['name'][:5] in pre:
                 brand.append(spe_name[data[i]['result']['name'][:5]])
            elif data[i]['result']['name'][:4] in pre:
                brand.append(spe_name[data[i]['result']['name'][:4]])
            elif data[i]['result']['name'][:3] in pre:
                brand.append(spe_name[data[i]['result']['name'][:3]])
            elif data[i]['result']['name'][:2] in pre:
                brand.append(spe_name[data[i]['result']['name'][:2]])

            else:
                name = data[i]['result']['name'].split('鍋')
                if len(name) == 1:
                    brand.append(re.split('/|【|-|,|;', name[0])[0])
                else:
                    brand.append(re.split('/|【|-|,|;', name[0])[0]+'鍋')
        except:
            continue
                

tmp = list(set(brand))
hot_pot_brand = {
    "品牌": [tmp[i] for i in range(len(tmp))]
}
df_brand = pd.DataFrame(hot_pot_brand)

In [7]:
df_brand

,品牌
0,官也溫菜石頭火鍋
1,12MINI快煮鍋
2,肉多多火鍋
3,灰鍋
4,鮨一的鍋
...,...
493,钱都日式涮涮锅
494,一番亭涮涮鍋
495,麻辣45 本店
496,巿民大道涮涮鍋


In [8]:
#服務table
lng = []
lat = []
tel = []
for k in os.listdir('./json'):
    if k!= '.ipynb_checkpoints':
        with open('./json/{}'.format(k)) as f:
            data = json.load(f)
    for i in range(len(data)):
        try:
            tel.append(data[i]['result']['formatted_phone_number'])
            lng.append(data[i]['result']['geometry']['location']['lng'])
            lat.append(data[i]['result']['geometry']['location']['lat'])
        except:
#             print(k,i)
            continue

service =  {'經度':lng,
            '緯度':lat,
            '電話':tel
    
}
df_service = pd.DataFrame(service)

In [9]:
df_service

,經度,緯度,電話
0,121.574922,25.045698,02 2766 2966
1,121.578859,25.041645,02 8785 1207
2,121.566755,25.039532,02 2723 7708
3,121.568109,25.035408,02 2345 6988
4,121.579927,25.040221,02 2726 2095
...,...,...,...
627,121.510848,25.050372,02 2555 7606
628,121.509354,25.053420,02 2558 9991
629,121.511150,25.065249,02 2597 9847
630,121.515776,25.071664,02 2595 7611


In [10]:
#評論table
comment1 = []
comment2 = []
comment3 = []
tel = []
for k in os.listdir('./json'):
    if k!= '.ipynb_checkpoints':
        with open('./json/{}'.format(k)) as f:
            data = json.load(f)
    for i in range(len(data)):
        try:
            tel.append(data[i]['result']['formatted_phone_number'])
            try:
                if len(data[i]['result']['reviews']) >= 3:
                    comment1.append(data[i]['result']['reviews'][0]['text'])
                    comment2.append(data[i]['result']['reviews'][1]['text'])
                    comment3.append(data[i]['result']['reviews'][2]['text'])
                elif len(data[i]['result']['reviews']) == 2:
                    comment1.append(data[i]['result']['reviews'][0]['text'])
                    comment2.append(data[i]['result']['reviews'][1]['text'])
                    comment3.append(None)
                elif len(data[i]['result']['reviews']) == 1:
                    comment1.append(data[i]['result']['reviews'][0]['text'])
                    comment2.append(None)
                    comment3.append(None)
            except:
                comment1.append(None)
                comment2.append(None)
                comment3.append(None)
        except:
            continue

            

comment =  {'評論人1':comment1,
            '評論人2':comment2,
            '評論人3':comment3,
            '電話':tel
    
}
df_comment = pd.DataFrame(comment)

In [11]:
df_comment

,評論人1,評論人2,評論人3,電話
0,Yummy,Excellent ! Shrimps are huge…,Nice,02 2766 2966
1,Great hotpot place. Very clean. Very friendly....,"Good food, fair price and amazing service.","Friendly staff, good ambience, decent food, re...",02 8785 1207
2,Was a fan but NOT ANYMORE. This was my top cho...,"Yesterday, a group of people went to Xinyi sto...","Located in Xinyi District, Xinguang Mitsukoshi...",02 2723 7708
3,"Best hotpot ever! Wonderful food, great servic...",Delicious food with decent service and relaxin...,Nice place to have a cozy meal in a cold cold ...,02 2345 6988
4,Nice staff!,"The ingredients in the pot can be richer, in t...",用foodpanda點外送，今天點的是招牌豚骨鍋搭牛肉，不想要火鍋料，想要換成豆皮，老闆真的...,02 2726 2095
...,...,...,...,...
627,Service is really good! Overall quite satisfie...,Good!!,The weather is getting colder and colder. If y...,02 2555 7606
628,"One of my favorite hot pots in all of Taipei, ...",Awesome food with nice old ambience,Loved the herbal spicy soup!! Its amazingly go...,02 2558 9991
629,Good,Great!!,The service staff here are super kind\nI am a ...,02 2597 9847
630,Good and reasonably priced to fill yourself up...,Good!,內用進店家確認人數\n會給空碗\n自行選找位置\n用餐完畢在結帳即可\n洗手間在地下一樓最裡...,02 2595 7611


In [12]:
#營業時間table
Day, open1, close1 = [],[],[]
tel = []
for k in os.listdir('./json'):
    if k!= '.ipynb_checkpoints':
        with open('./json/{}'.format(k)) as f:
            data = json.load(f)
    for i in range(len(data)):
        for j in data[i]['result']['opening_hours']['periods']:
            try:
                tel.append(data[i]['result']['formatted_phone_number'])
                Day.append(j['close']['day'])
                open1.append(int(j['open']['time']))
                close1.append(int(j['close']['time']))
            except:
                #print(k,i)
                continue
                
operation_time =  {'營業星期':Day,
        '開門時間':open1,
        '關門時間':close1,
        '電話':tel
}   
df_operation = pd.DataFrame(operation_time)

In [13]:
df_operation 

,營業星期,開門時間,關門時間,電話
0,0,1130,1430,02 2766 2966
1,0,1700,2230,02 2766 2966
2,1,1130,1430,02 2766 2966
3,1,1700,2230,02 2766 2966
4,2,1130,1430,02 2766 2966
...,...,...,...,...
5847,2,1100,2100,02 2599 5259
5848,3,1100,2100,02 2599 5259
5849,4,1100,2100,02 2599 5259
5850,5,1100,2100,02 2599 5259


In [14]:
with open('hot_pot_schema.sql') as f:
    create_db_sql = f.read()
    
#建立 hot_pot.db 並把 table 的 schema 匯入進去
db = sqlite3.connect('hot_pot.db')
with db: 
    db.executescript(create_db_sql)
with db:
    db.executemany(
      'INSERT INTO  Store (Tel, Addr, Comment_num, Price_level, Store_name, Brand, City, District, Avg_rating) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)',
        df_store.values)
    db.executemany(
      'INSERT INTO  Region (City, District) VALUES (?, ?)',
        df_region.values)
    db.executemany(
      'INSERT INTO  Hot_pot (Brand) VALUES (?)',
        df_brand.values)
    db.executemany(
      'INSERT INTO  Service (Lng, Lat, Tel) VALUES (?, ?, ?)',
        df_service.values)
    db.executemany(
      'INSERT INTO  Commentor (Commentor1, Commentor2, Commentor3, Tel) VALUES (?, ?, ?, ?)',
        df_comment.values)
    db.executemany(
      'INSERT INTO  Operation (Open_Day, Begin_, End_, Tel) VALUES (?, ?, ?, ?)',
        df_operation.values)

In [16]:
with db:
    db.execute('PRAGMA foreign_keys = ON')

In [17]:
with db:
    c = db.execute('SELECT * FROM Hot_pot Limit 5')

In [18]:
for i in c:
    print(i)

('官也溫菜石頭火鍋',)
('12MINI快煮鍋',)
('肉多多火鍋',)
('灰鍋',)
('鮨一的鍋',)


In [ ]:
# #開啟foreign key的功能(!必要步驟 否則cascade不會作用)
# with db:
#     db.execute('PRAGMA foreign_keys = ON')

In [ ]:
# #Update
# with db:
#     c = db.execute(" Update Hot_pot Set Brand='Mala Hot Pot lol' Where Brand='Mala Hot Pot' " )

In [19]:
# #Delete
# with db:
#     c = db.execute(" Delete From Hot_pot where Brand='Mala Hot Pot' " )

In [28]:
# #Insert
# with db:
#     db.execute("Insert Into Hot_pot (Brand) Values ('Mala Hot Pot')")
#     db.execute(" Insert Into Store (Tel, Addr, Comment_num, Price_level, Store_name, Brand, City, District, Avg_rating) \
#     Values ('112', 'lol', '0', '2', 'Male','Mala Hot Pot','台北市','北投區', 3.5)" )
#     db.execute("Insert Into Service (Lng, Lat, tel) Values (121, 20, '112')")
#     db.execute("Insert Into Commentor (Commentor1, Commentor2, Commentor3,tel) Values ('1', '2', '3', '112')")
#     db.execute("Insert Into Operation (Open_Day, Begin_, End_, tel) Values (0, 0000, 2400, '112')")
    

In [47]:
# with db:
#     c = db.execute(" Update Hot_pot Set Brand='Mala Hot Pot' Where Brand='Mala Hot Pot' " )

In [52]:
# with db:
#     c = db.execute("SELECT * FROM Hot_pot Where Brand='Mala Hot Pot' ")

In [50]:
# with db:
#     c = db.execute("SELECT * FROM Store Where Tel='112' ")

In [48]:
# with db:
#     c = db.execute("SELECT * FROM Store Where Brand='Mala Hot Pot' ")

In [53]:
# for i in c:
#     print(i)

('Mala Hot Pot',)


In [ ]:
########以下不用管

In [ ]:
# set(df_operation.電話) - set(df_store.電話)

In [ ]:
# from collections import Counter
# [k for k,v in Counter(list(df_comment.電話)).items() if v>1]

In [ ]:
# with open('./json/Wenshan.json') as f:
#     data = json.load(f)
# data[46]